In [503]:
# IMPORT LIBRARIES
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns 

from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing, metrics, tree, svm
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier, export_graphviz, DecisionTreeRegressor
from sklearn.externals.six import StringIO
from sklearn.metrics import classification_report, confusion_matrix, jaccard_similarity_score, log_loss, f1_score, mean_squared_error, mean_absolute_error 
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [611]:
# Import Dataframe
df = pd.read_csv('df_FeatureSelection.csv')

In [514]:
df.drop('SalePrice', axis = 1, inplace = True)

# DROP VARIABLES WITH LARGE VIF FACTOR OR LACK OF OBSERVATIONS
df.drop(['PoolArea', 'Exterior1st_CBlock', 'Exterior1st_VinylSd', 'SaleType_New', 
         'SaleCondition_Partial', 'GarageType_BuiltIn', 'Electrical_FuseA', 'Heating_GasA', 
         'ExterQual_TA', 'ExterCond_TA', 'Condition1_Norm','RoofStyle_Gable', 'RoofMatl_CompShg', 
         'Functional_Typ', 'HouseStyle_2Story', 'Exterior1st_ImStucc', 'Exterior1st_AsphShn', 
         'ExterCond_Po', 'Exterior1st_BrkComm'], axis = 1, inplace = True)

# DROP VARIABLES WITH LARGE VIF FACTOR (FourSquare Features)
'''
From the ranking system, we see that neighborhoods with many restaurants, tend to have many stores, nightlife, services 
and grocery options. They could therefore be argued to capture the same information about 'inner city housing'
We will drop stores, nightlife, grocery and services.
'''
df.drop(['stores', 'nightlife', 'grocery', 'service'], axis = 1, inplace = True)

In [537]:
# Create X and y dataframes
X = df.iloc[:,1:]
X.drop(['SalePrice_log'], axis = 1, inplace = True)
y = df['SalePrice_log'].values

fs_api =  ['restaurants', 'transport', 'school', 'golf', 'park', 
           'recreational', 'cultural', 'leisure']

# Create two sample sets to compare, one without the Foursquare API data, and one without the Neighborhood columns
X = X[X.columns.drop(list(X.filter(regex='Neighborhood')))]

cnames = X.columns

# STANDARDIZE
X = preprocessing.StandardScaler().fit(X).transform(X)

# Split into test and train set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Linear Regression

In [587]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

X_lin = pd.DataFrame(X_train)
X_lin.columns = cnames

X_lin = sm.add_constant(X_lin)
model = sm.OLS(y_train,X_lin)
results = model.fit()
results_summary = results.summary()

# Convert sm table to pandas df
results_as_html = results_summary.tables[1].as_html()
res_df = pd.read_html(results_as_html, header=0, index_col=0)[0]

In [590]:
res_df.loc[fs_api,:]

,coef,std err,t,P>|t|,[0.025,0.975]
restaurants,-0.0125,0.011,-1.094,0.274,-0.035,0.010
transport,-0.0462,0.011,-4.022,0.000,-0.069,-0.024
school,0.0567,0.014,4.104,0.000,0.030,0.084
golf,-0.0002,0.006,-0.027,0.979,-0.012,0.012
park,0.0444,0.010,4.491,0.000,0.025,0.064
recreational,-0.0289,0.008,-3.828,0.000,-0.044,-0.014
cultural,-0.0137,0.010,-1.408,0.159,-0.033,0.005
leisure,0.0288,0.008,3.702,0.000,0.014,0.044


In [526]:
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
vif_series = pd.Series([vif(X_lin.values, i) 
               for i in range(X_lin.shape[1])], 
              index=X_lin.columns)
vif_series

const                     1.043760
LotFrontage               2.423157
LotArea                   3.530764
OverallQual               4.849893
OverallCond               2.586546
YearRemodAdd              3.390467
MasVnrArea                3.002730
BsmtQual                  4.602003
BsmtCond                  2.069923
BsmtExposure              1.760244
TotalBsmtSF               6.042632
HeatingQC                 2.716219
LowQualFinSF              2.147476
GrLivArea                11.350008
FullBath                  2.832632
BedroomAbvGr              3.161269
KitchenQual               3.009547
TotRmsAbvGrd              5.934338
Fireplaces                4.409977
FireplaceQu               4.524157
GarageCars                7.950517
GarageArea                7.360771
GarageQual                2.629626
WoodDeckSF                1.454503
OpenPorchSF               1.550347
EnclosedPorch             1.651807
3SsnPorch                 1.186935
ScreenPorch               1.304572
PoolQC              

## Spot check algorithms using cross-validation

In [527]:
pipelines = []
pipelines.append(('LinearRegression', Pipeline([('LR',LinearRegression())])))
pipelines.append(('LassoRegression', Pipeline([('LASSO', Lasso())])))
pipelines.append(('ElasticNet', Pipeline([('EN', ElasticNet())])))
pipelines.append(('RandomForest', Pipeline([('RF', RandomForestRegressor())])))
pipelines.append(('SupportVector', Pipeline([('SVR', SVR())])))
pipelines.append(('GradientBoosting', Pipeline([('GBM', GradientBoostingRegressor())])))

kfold = KFold(n_splits=10)

results = []
names = []

for name, model in pipelines:
    
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring='neg_mean_absolute_error')
    results.append(cv_results)
    names.append(name)
    #print(cv_results)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

LinearRegression: -3626332352.276269 (10878997056.468193)
LassoRegression: -0.318790 (0.017632)
ElasticNet: -0.318790 (0.017632)
RandomForest: -0.105817 (0.006407)
SupportVector: -0.127016 (0.014782)
GradientBoosting: -0.096784 (0.005315)


As can be seen from the neg_mean_absolute_error we observe a large error in the linear regression. This could be an effect of breaches to the assumptions underlying the regression model. Most likely highly correlated variables.

Lasso and Elastic net makes more sense than a linear regression as we have a normalization effect.

A random forest approach makes a lot of sense since we have a lot of binary variables (0-1 features).

We see that support vectors, random forest and gradient boosting regression models perform better than the rest on the train set.

As Random Forest is a tree-based model, we have a problem with our many categorical variables. From a splitting algorithms POW, all binary features are independent, and as a result, continuous variables are assigned higher importance and chosen as the top tree splits. One could alleviate this issue by using a H2o framework rather than SkLearn since H2o does not require categorical features to be one-hot encoded. This potential improvement is considered out of the scope of this project.

## Model Tuning

#### Random Forest Tuning

In [591]:
parameters = {'n_estimators': range(100, 300, 20), 
              'max_depth': range(20, 40, 5)
             }

rf = RandomForestRegressor(criterion = 'mae')

np.random.seed(1)
gs_rf = GridSearchCV(rf, parameters, scoring = 'neg_mean_absolute_error', cv = kfold)
gs_rf.fit(X_train, y_train)

GridSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=False),
             error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mae', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=None,
                                             v

In [595]:
# Cross validation score
print(gs_rf.best_estimator_)
print(gs_rf.best_score_)

# Neg Mean Abs Error
y_pred_rf = np.expm1(gs_rf.best_estimator_.predict(X_train))
nmae_rf = -mean_absolute_error(np.expm1(y_train), y_pred_rf)
print(nmae_rf)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mae',
                      max_depth=25, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=280, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)
-0.10403987832041509
-7581.398827834744


280 estimators and max depth of 25 are found to be optimal. The tuned model performs slightly better than 0.105817 by the not-tuned model.

#### Support Vector Tuning

In [597]:
parameters = {'C': [0.1, 0.5, 1, 1.5, 2], 
              'kernel': ('linear', 'sigmoid', 'rbf', 'poly')
             }

sv = SVR()
gs_sv = GridSearchCV(sv, parameters, scoring = 'neg_mean_absolute_error', cv = kfold)
gs_sv.fit(X_train, y_train)

GridSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=False),
             error_score=nan,
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='scale', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.1, 0.5, 1, 1.5, 2],
                         'kernel': ('linear', 'sigmoid', 'rbf', 'poly')},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_absolute_error', verbose=0)

In [601]:
# Cross validation score
print(gs_sv.best_score_)
print(gs_sv.best_estimator_)

# Neg Mean Abs Error
y_pred_sv = np.expm1(gs_sv.best_estimator_.predict(X_train))
nmae_sv = -mean_absolute_error(np.expm1(y_train), y_pred_sv)
print(nmae_sv)

-0.09912600866880174
SVR(C=0.5, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
-17709.827784055844


The tuned support vector performs much better from 0.127016 with the un-tuned version. The optimal C is found to be 0.5, and a linear kernel are optimal.

#### Gradient Boosting Tuning

In [602]:
parameters = {'learning_rate': [0.01, 0.05, 0.1, 0.15], 
              'n_estimators': range(100, 200, 10),
              'loss': ('ls', 'lad', 'huber', 'quantile')
             }

gb = GradientBoostingRegressor()
gs_gb = GridSearchCV(gb, parameters, scoring = 'neg_mean_absolute_error', cv = kfold)
gs_gb.fit(X_train, y_train)

GridSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=False),
             error_score=nan,
             estimator=GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                 criterion='friedman_mse',
                                                 init=None, learning_rate=0.1,
                                                 loss='ls', max_depth=3,
                                                 max_features=None,
                                                 max_leaf_nodes=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_w...
                                                 random_state=None,
                                             

In [603]:
print(gs_gb.best_estimator_)

# Cross validation score
print(gs_gb.best_score_)

# Neg Mean Abs Error
y_pred_gb = np.expm1(gs_gb.best_estimator_.predict(X_train))
nmae_gb = -mean_absolute_error(np.expm1(y_train), y_pred_gb)
print(nmae_gb)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='huber',
                          max_depth=3, max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=180,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)
-0.0932840275546446
-9391.55721358478


A learning rate of 0.1, and a loss function set to huber with 180 estimators are optimal.

## Test ensembles

In [604]:
tpred_rf = np.expm1(gs_rf.best_estimator_.predict(X_test))
tpred_sv = np.expm1(gs_sv.best_estimator_.predict(X_test))
tpred_gb = np.expm1(gs_gb.best_estimator_.predict(X_test))

In [606]:
y_test_exp = np.expm1(y_test)

nmae_test_rf = -mean_absolute_error(y_test_exp, tpred_rf)
nmae_test_sv = -mean_absolute_error(y_test_exp, tpred_sv)
nmae_test_gb = -mean_absolute_error(y_test_exp, tpred_gb)
nmae_test_comb = -mean_absolute_error(y_test_exp, (tpred_rf + tpred_sv + tpred_gb) / 3)

print('rf mae:', nmae_test_rf)
print('sv mae:', nmae_test_sv)
print('gb mae:', nmae_test_gb)
print('combined mae:', nmae_test_comb)

rf mae: -17262.53153551469
sv mae: -15930.752617648075
gb mae: -15504.138400427957
combined mae: -14335.53986195571


Interestingly, this is a case where 1+1 = 3 as the average of the three models performs much better than each individual model. The optimal model is therefore a combination of the three.

## Model Evaluation

In [607]:
eval_df = ([nmae_rf, nmae_test_rf], [nmae_sv, nmae_test_sv], [nmae_gb, nmae_test_gb], ['', nmae_test_comb])
pd.DataFrame(eval_df, columns = ['Train Neg MAE','Test Neg MAE'], 
             index = ['Random Forest', 'Support Vector', 'Gradient Boosting', 'Combined'])

,Train Neg MAE,Test Neg MAE
Random Forest,-7581.4,-17262.531536
Support Vector,-17709.8,-15930.752618
Gradient Boosting,-9391.56,-15504.138400
Combined,,-14335.539862


## Feature importance RF

In [610]:
f_imp = gs_rf.best_estimator_.feature_importances_

cn = df.iloc[:,1:]
cn.drop('SalePrice_log', axis = 1, inplace = True)
f_names = cn.columns

f_df = pd.concat([pd.Series(f_names), pd.Series(f_imp)], axis = 1)
f_df.columns = ['Feature', 'Variable Importance']
f_df.sort_values('Variable Importance', ascending = False)[0:25]

,Feature,Variable Importance
2,OverallQual,0.330655
12,GrLivArea,0.126630
9,TotalBsmtSF,0.088130
20,GarageArea,0.045721
19,GarageCars,0.040806
38,age,0.032982
1,LotArea,0.030514
4,YearRemodAdd,0.020636
0,LotFrontage,0.020093
13,FullBath,0.015834


As a final remark, we observe that overall quality, 'Rates the overall material and finish of the house', appear to be the most important feature according to our random forest model, followed by GrLivArea: 'Above grade (ground) living area square feet'. The first Foursquare API feature to appear is 'Restaurants' in rank 21.  